In [14]:
from config import *
import pandas as pd
import seaborn as sns
from datetime import datetime
import calendar 
import numpy as np

In [6]:
df_accident = pd.read_csv(CFG.data_rootfolder / "ACCIDENT.CSV")
df_accident.head()

/var/folders/lg/ssxqzhrj33g6tvlvnhwsmxxr0000gn/T/ipykernel_12823/206710196.py:1: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accident = pd.read_csv(CFG.data_rootfolder / "ACCIDENT.CSV")


,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,...,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,...,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,...,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,...,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,...,3,0,3,0,0,1,5,Not at intersection,3,50


In [15]:
#Some columns are used to describe some encoded columns, let's get extract the mapping
mapping_df_accident = {}

encoded_columns = ["ACCIDENT_TYPE","DAY_OF_WEEK", "DCA_CODE",'LIGHT_CONDITION','ROAD_GEOMETRY']
description_columns = ["Accident Type Desc", 'Day Week Description',"DCA Description", 'Light Condition Desc', "Road Geometry Desc"]

mapping_df_accident = get_mapping_encoded_values(df_accident, encoded_columns=encoded_columns, description_columns=description_columns)
mapping_df_accident

{'ACCIDENT_TYPE': {1: 'Collision with vehicle',
  7: 'Fall from or in moving vehicle',
  4: 'Collision with a fixed object',
  2: 'Struck Pedestrian',
  5: 'collision with some other object',
  6: 'Vehicle overturned (no collision)',
  8: 'No collision and no object struck',
  3: 'Struck animal',
  9: 'Other accident'},
 'DAY_OF_WEEK': {6: 'Saturday',
  7: 'Saturday',
  1: 'Sunday',
  2: 'Tuesday',
  3: 'Wednesday',
  4: 'Wednesday',
  5: 'Friday',
  0: 'Sunday'},
 'DCA_CODE': {113: 'RIGHT NEAR (INTERSECTIONS ONLY)               ',
  190: 'FELL IN/FROM VEHICLE                    ',
  130: 'REAR END(VEHICLES IN SAME LANE)               ',
  121: 'RIGHT THROUGH                        ',
  116: 'LEFT NEAR (INTERSECTIONS ONLY)               ',
  171: 'LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE       ',
  140: 'U TURN                           ',
  109: 'ANY MANOEUVRE INVOLVING PED NOT INCLUDED IN DCAs 100-108.  ',
  173: 'RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE      ',
  110:

In [8]:
# To deal with time data, we'll create some categorical features : hour of the day when the accident happened [ACCIDENTHOUR], day of the year when the accident happened [ACCIDENTDAYNUMBER].

df_accident["ACCIDENTDATETIME"] = df_accident["ACCIDENTDATE"] + " " + df_accident["ACCIDENTTIME"]
df_accident["ACCIDENTDATETIME"] = pd.to_datetime(df_accident["ACCIDENTDATETIME"]) #It takes a bit of time because we let it guess the format as there is diferent formats(%Y-%m-%d %H:%M:%S",%Y/%m/%d %H:%M:%S" ...)


# To keep day number of the year constant, we'll watch out for peak years and 29th of february
def day_nb_of_the_year(date: datetime) -> int: 
    raw_day_number = int(date.strftime('%j')) 
    
    if calendar.isleap(date.year) and raw_day_number >= 60 : 
        if  raw_day_number == 60 : # 29/02/xxxx
            return 366 
        else :
            return raw_day_number -1 
    return raw_day_number
         
df_accident["ACCIDENTDAYNUMBER"] = df_accident["ACCIDENTDATETIME"].apply(lambda x: day_nb_of_the_year(x))    
df_accident["ACCIDENTHOUR"] = df_accident["ACCIDENTDATETIME"].apply(lambda x: x.hour)

#df_accident["ACCIDENTDAYNUMBER"].value_counts().sort_index().plot()
#df_accident["ACCIDENTHOUR"].value_counts().sort_index().plot()

#df_accident.head()


In [9]:
redundant_columns = ["NO_PERSONS_NOT_INJ"]
encoded_columns = ["ACCIDENTTIME", "ACCIDENTDATE", "ACCIDENTDATETIME"]
discarded_columns = ["EDITION", "PAGE", "DIRECTORY", "GRID_REFERENCE_X", "GRID_REFERENCE_Y" ]

df = df_accident.drop(columns = discarded_columns + description_columns + encoded_columns + redundant_columns).set_index("ACCIDENT_NO")

df.head()

,ACCIDENT_TYPE,DAY_OF_WEEK,DCA_CODE,LIGHT_CONDITION,NODE_ID,NO_OF_VEHICLES,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,POLICE_ATTEND,ROAD_GEOMETRY,SEVERITY,SPEED_ZONE,ACCIDENTDAYNUMBER,ACCIDENTHOUR
ACCIDENT_NO,,,,,,,,,,,,,,,,
T20060000010,1,6,113,1,43078,3,6,0,1,0,1,1,3,60,13,12
T20060000018,1,6,113,1,29720,2,4,0,1,0,1,2,3,70,13,19
T20060000022,7,7,190,1,203074,1,2,1,0,0,1,5,2,100,14,12
T20060000023,1,7,130,1,55462,2,2,1,0,0,1,2,2,80,14,11
T20060000026,1,7,121,1,202988,2,3,0,3,0,1,5,3,50,14,10


In [10]:
df_person = pd.read_csv(CFG.data_rootfolder / "PERSON.CSV")
df_person.head(20)

/var/folders/lg/ssxqzhrj33g6tvlvnhwsmxxr0000gn/T/ipykernel_12823/1136902260.py:1: DtypeWarning: Columns (6,9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_person = pd.read_csv(CFG.data_rootfolder / "PERSON.CSV")


,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE,Age Group,INJ_LEVEL,Inj Level Desc,SEATING_POSITION,HELMET_BELT_WORN,ROAD_USER_TYPE,Road User Type Desc,LICENCE_STATE,PEDEST_MOVEMENT,POSTCODE,TAKEN_HOSPITAL,EJECTED_CODE
0,T20060000010,01,A,F,NaN,unknown,4,Not injured,LF,1,3,Passengers,,0,3130.0,,0
1,T20060000010,02,C,M,43.0,40-49,4,Not injured,LF,1,3,Passengers,,0,7310.0,,0
2,T20060000010,03,C,M,22.0,22-25,4,Not injured,LR,1,3,Passengers,,0,7310.0,,0
3,T20060000010,A,A,M,72.0,70+,4,Not injured,D,1,2,Drivers,V,0,3130.0,,0
4,T20060000010,B,B,F,62.0,60-64,3,Other injury,D,1,2,Drivers,V,0,NaN,N,0
5,T20060000010,C,C,M,39.0,30-39,4,Not injured,D,1,2,Drivers,V,0,3804.0,,0
6,T20060000018,01,B,F,18.0,17-21,4,Not injured,CR,9,3,Passengers,V,0,3175.0,,0
7,T20060000018,02,B,M,18.0,17-21,4,Not injured,RR,1,3,Passengers,,0,NaN,,0
8,T20060000018,A,A,M,30.0,30-39,4,Not injured,D,1,2,Drivers,V,0,3782.0,,0
9,T20060000018,B,B,F,20.0,17-21,3,Other injury,D,1,2,Drivers,V,0,3805.0,N,0


[]

,ACCIDENT_TYPE,DAY_OF_WEEK,DCA_CODE,LIGHT_CONDITION,NODE_ID,NO_OF_VEHICLES,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,POLICE_ATTEND,ROAD_GEOMETRY,SEVERITY,SPEED_ZONE,ACCIDENTDAYNUMBER,ACCIDENTHOUR
ACCIDENT_NO,,,,,,,,,,,,,,,,
T20060000010,1,6,113,1,43078,3,6,0,1,0,1,1,3,60,13,12
T20060000018,1,6,113,1,29720,2,4,0,1,0,1,2,3,70,13,19
T20060000022,7,7,190,1,203074,1,2,1,0,0,1,5,2,100,14,12
T20060000023,1,7,130,1,55462,2,2,1,0,0,1,2,2,80,14,11
T20060000026,1,7,121,1,202988,2,3,0,3,0,1,5,3,50,14,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T20200019239,1,0,142,1,665835,2,4,1,0,0,1,5,2,50,11,12
T20200019247,4,1,171,1,665912,1,2,2,0,0,1,5,2,999,11,15
T20200019250,1,0,116,1,226603,2,2,1,0,0,1,1,2,60,11,18
